# Rift Data Academy

Premier *mid laner* server KR : 胡小青超酷#douyu  
Enregistré le 15/10/2024, rang général 2ième.   
puuid : yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A  
Joueur pro : Qing  

In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import dotenv_values
import sqlite3


In [2]:
mid_kr_puuid = "yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A"

In [3]:
config = dotenv_values(".env")
api_key = config['API_KEY']

In [4]:
requests.get(f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A/ids?queue=420&start=0&count=100&api_key={api_key}").json()

['KR_7329210097',
 'KR_7329178248',
 'KR_7329125077',
 'KR_7329046760',
 'KR_7329004245',
 'KR_7328931883',
 'KR_7328873415',
 'KR_7328828963',
 'KR_7328802260',
 'KR_7328760490',
 'KR_7328710571',
 'KR_7328682718',
 'KR_7328642296',
 'KR_7328580852',
 'KR_7328542031',
 'KR_7328490197',
 'KR_7328179457',
 'KR_7328156965',
 'KR_7328149824',
 'KR_7328127527',
 'KR_7328106754',
 'KR_7328077295',
 'KR_7328049584',
 'KR_7328002222',
 'KR_7327450345',
 'KR_7327420424',
 'KR_7327396894',
 'KR_7327368474',
 'KR_7327333878',
 'KR_7327314771',
 'KR_7327291661',
 'KR_7327198954',
 'KR_7327029255',
 'KR_7327001968',
 'KR_7326982806',
 'KR_7326959717',
 'KR_7326921739',
 'KR_7326886546',
 'KR_7326828067',
 'KR_7326759206',
 'KR_7326715474',
 'KR_7326626776',
 'KR_7326575540',
 'KR_7326538030',
 'KR_7326483526',
 'KR_7326438909',
 'KR_7326390541',
 'KR_7326368039',
 'KR_7326321624',
 'KR_7325930087',
 'KR_7325909890',
 'KR_7325905779',
 'KR_7325902435',
 'KR_7325898209',
 'KR_7325895228',
 'KR_73258

In [6]:
conn = sqlite3.connect('koren_soloq_db.db')
cursor = conn.cursor()
conn.execute('''
    CREATE TABLE IF NOT EXISTS MATCHID (
        id TEXT PRIMARY KEY UNIQUE 
    )
''')
conn.close()

In [11]:
#stocker tous les id dans la table matchid